# Projet maison n° 3

In [75]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [76]:
# names us
from sqlite3 import DateFromTicks


def df_names_us():
    df = pd.read_csv("names/yob1880.txt", header=None, names=["name", "gender", "births"])
    df["year"] = 1880
    for year in range(1881, 2022):
        actual = pd.read_csv(f"names/yob{year}.txt", header=None, names=["name", "gender", "births"])
        actual["year"] = year
        df = pd.concat([df, actual])
    df = df.reset_index(drop=True)
    df = df[["year", "name", "gender", "births"]]
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [77]:
# names fr
def df_names_fr():
    df = pd.read_csv("nat2021.csv", delimiter=";", keep_default_na=False, na_values=['_PRENOMS_RARES', 'XXXX'])
    df.rename(columns={'annais': 'year',
                   'preusuel': 'name',
                   'sexe': 'gender',
                   'nombre': 'births'},
          inplace=True, errors='raise')
    df = df[["year", "name", "gender", "births"]]
    df = df.loc[df["name"].str.len() >= 2].copy()
    df["name"] = df["name"].str.title()
    df["gender"] = df["gender"].map({1: "M", 2: "F"})
    df.dropna(inplace=True)
    df.reset_index(inplace=True, drop = True)
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [78]:
import re
import numpy as np
# taux de change
def df_taux_change(devises= None):
    df = pd.read_csv("../Kit-Big-Data/session3/Webstat_Export.csv", delimiter=";", na_values=['-'])
    df = df[5:]

    df.rename(columns={"Titre :": "date"}, inplace = True)
    df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")
    df = df.set_index('date')

    col = df.columns
    codes = [re.search('\((.+?)\)', name).group(1) for name in col]
    df.columns = codes

    df = df[devises]

    df.sort_index(inplace=True)
    df.fillna(method="ffill", inplace=True)  
    df = df.applymap(lambda x: x.replace(",", "."))
    df = df.astype("float")
    return df

In [79]:
df = df_taux_change(["CHF", "GBP", "USD"])
display(df.head(10))


,CHF,GBP,USD
date,,,
1999-01-04,1.6168,0.71110,1.1789
1999-01-05,1.6123,0.71220,1.1790
1999-01-06,1.6116,0.70760,1.1743
1999-01-07,1.6165,0.70585,1.1632
1999-01-08,1.6138,0.70940,1.1659
1999-01-09,1.6138,0.70940,1.1659
1999-01-10,1.6138,0.70940,1.1659
1999-01-11,1.6104,0.70440,1.1569
1999-01-12,1.6072,0.70660,1.1520


In [80]:
df.dtypes

CHF    float64
GBP    float64
USD    float64
dtype: object

### Tests

In [81]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [82]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=3)
    runner.run(test_suite)

In [83]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.581s

OK
